In [ ]:
from appgeopy import *
from my_packages import *

In [1]:
# **2025/3/24**

Prepare monthly data, ready for LOS decomposition

# ```{python}
"""
General Description:
This snippet reads a pickled time-series dataset, filters displacement columns,
resamples them to monthly intervals, and re-combines the results with ancillary
information and geometry data.
"""

# Specify the path to the pickled DataFrame
pickle_fpath = r"DESC_overlap_asc_+pointkey_TWD97.pkl"

# Load the DataFrame with zip compression
timeseries_full = pd.read_pickle(pickle_fpath, compression="zip")

# Collect displacement columns (those that start with 'D')
cumdisp_columns = [col for col in timeseries_full.columns if col.startswith("D")]
# Extract only displacement columns
cumdisp_timeseries = timeseries_full.loc[:, cumdisp_columns]
# Convert displacement columns to DateTime objects (removing the "D" prefix)
cumdisp_timeseries.columns = [pd.to_datetime(col[1:]) for col in cumdisp_timeseries.columns]
# Resample displacement data monthly and compute mean
monthly_resampled_cumdisp_timeseries = cumdisp_timeseries.resample("MS", axis=1).mean()
# Rename columns with "D" prefix and YYYYMMDD format
monthly_resampled_cumdisp_timeseries.columns = [
    "D" + col.strftime("%Y%m%d") for col in monthly_resampled_cumdisp_timeseries.columns
]

# Separate non-displacement columns
info_df = timeseries_full.loc[:, [col for col in timeseries_full.columns if not col.startswith("D")]]

# Concatenate info (excluding the last column), resampled displacements, and geometry
final_timeseries_full = pd.concat(
    [
        info_df.iloc[:, :-1],
        monthly_resampled_cumdisp_timeseries,
        info_df.loc[:, ["geometry"]],
    ],
    axis=1,
)

# Display the final DataFrame
# final_timeseries_full.to_pickle("MonthlyResample"+pickle_fpath, compression='zip')
# ```

SyntaxError: invalid syntax (2824709791.py, line 1)

**2025/3/24**

Extract the incidence and azimuth angles for ASC and DESC, respectively

```{python}
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.transform import rowcol
from joblib import Parallel, delayed

start = time.time()

# Load point shapefile
point_fpath = r"D:\1000_SCRIPTS\003_Project002\20250222_GTWR001\1_PrepareDatasets\3_InSAR\asc_run003L\output_shp\ASC_overlap_desc_+pointkey_TWD97.shp"
direction = os.path.basename(point_fpath).split("_")[0]

raster_filelist = [
    r"D:\1000_SCRIPTS\003_Project002\20250222_GTWR001\1_PrepareDatasets\3_InSAR\asc_run003L\raster_output\asc_inc_twd97.tif",
    r"D:\1000_SCRIPTS\003_Project002\20250222_GTWR001\1_PrepareDatasets\3_InSAR\asc_run003L\raster_output\asc_azim_twd97.tif"
]

# Load point shapefile
points = gpd.read_file(point_fpath)

for raster_fpath in raster_filelist:
    raster_base = os.path.basename(raster_fpath)

    if "inc" in raster_base:
        colname = f"{direction}_INC"
    elif "azim" in raster_base:
        colname = f"{direction}_AZIM"
    else:
        raise ValueError("No specific keyword in raster filename")

    # Read the raster into memory
    with rasterio.open(raster_fpath) as src:
        raster_array = src.read(1)
        transform = src.transform

    def get_raster_value(geom):
        try:
            row, col = rowcol(transform, geom.x, geom.y)
            if 0 <= row < raster_array.shape[0] and 0 <= col < raster_array.shape[1]:
                return raster_array[row, col]
        except:
            pass
        return np.nan

    # Parallel extraction using joblib
    raster_vals = Parallel(n_jobs=-1)(delayed(get_raster_value)(geom) for geom in points.geometry)

    # Assign results
    points[colname] = raster_vals

end = time.time()
elapsed_time_seconds = round((end - start), 2)
elapsed_time_minutes = round((elapsed_time_seconds / 60), 2)

print("\n\n" + "-" * 50)
print("Processing time (seconds): {}".format(elapsed_time_seconds))
print("Processing time (minutes): {}".format(elapsed_time_minutes))
```

**2025/3/25**

Combine incidence and azimuth angles from ascending & descending into one file

```{python}
asc_inc_azim = pd.read_pickle("ASC_inc_azim.pkl", compression='zip')
desc_inc_azim = pd.read_pickle("DESC_inc_azim.pkl", compression='zip')

asc_inc_azim = asc_inc_azim.sort_values(by="PointKey").reset_index(drop=True)
desc_inc_azim = desc_inc_azim.sort_values(by="PointKey").reset_index(drop=True)

combined = asc_inc_azim.copy()
combined.set_index("PointKey", inplace=True)
desc_inc_azim.set_index("PointKey",inplace=True)

for col in ["DESC_INC", "DESC_AZIM"]:
    combined[col] = combined.index.map(desc_inc_azim[col])

columns = combined.columns
new_cols = [columns[0]] + list(columns[1:][::-1])
combined = combined[new_cols]

combined.to_pickle(r"ASC_DESC_inc_azim.xz")
```